# Обучение модели

In [1]:
!git clone -b paper https://github.com/WongKinYiu/yolor
%cd yolor
#!pip install -qr requirements.txt
!pip install Cython
!pip install matplotlib>=3.2.2
!pip install numpy>=1.18.5
!pip install opencv-python>=4.1.2
!pip install Pillow
!pip install PyYAML>=5.3.1
!pip install scipy>=1.4.1
!pip install tensorboard>=1.5
!pip install torch==1.7.1+cu110 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torchvision==0.8.2+cu110 -f https://download.pytorch.org/whl/torch_stable.html
!pip install tqdm>=4.41.0
!pip install seaborn>=0.11.0
!pip install pandas
!pip install thop
!conda install --yes pycocotools
!conda install --yes cudatoolkit=11.0 -c pytorch
!git clone https://github.com/JunnYu/mish-cuda
%cd mish-cuda
!python setup.py build install
%cd ..
!git clone https://github.com/fbcotter/pytorch_wavelets
%cd pytorch_wavelets
!pip install .
%cd ..

fatal: целевой путь «yolor» уже существует и не является пустым каталогом.
/media/andrey/Новый том/depersonalization_object_detection/task/yolor
Looking in links: https://download.pytorch.org/whl/torch_stable.html
Looking in links: https://download.pytorch.org/whl/torch_stable.html
Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [1]:
import torch
import os
import random
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import KFold
import torch.backends.cudnn as cudnn
import yaml
from operator import itemgetter
from shutil import copyfile
from tqdm import tqdm
from IPython.display import Image, display
from IPython.core.magic import register_line_cell_magic
import imagehash
from PIL import Image, ImageFilter

In [2]:
!nvidia-smi

Tue Aug 16 20:48:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:05:00.0  On |                  N/A |
|  0%   50C    P8    34W / 350W |    289MiB / 24576MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda
NVIDIA GeForce RTX 3090
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [33]:
base_path = Path('/media/andrey/Новый том/depersonalization_object_detection/')
train_image_path = base_path / "train/images/"
train_labels_path = base_path / "train/labels/"
test_image_path = base_path / "test/"
yolor_data_path = base_path / "task/yolor/data/"
class_names = ['car', 'head', 'face', 'human', 'carplate']

In [34]:
def read_image(path: Path):
    image = cv2.imread(str(path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def show_image(name: str, image: np.ndarray):
    plt.figure(figsize=(10, 14))
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title(f"Image: {name}")

def init_torch_seeds(seed=0):
    torch.manual_seed = seed
    cudnn.deterministic = True
    cudnn.benchmark = False

def init_seeds(seed=0):
    random.seed = seed
    np.random.seed(seed)
    init_torch_seeds(seed)
    
def copy_files(images_name, image2dataset_type, labels):
    for img_path in tqdm(images_name):
        split_type = image2dataset_type[img_path]
        try:
            copyfile(img_path, os.path.join('./', split_type, img_path.split('/')[-1]))
            bbox_name = img_path.split('/')[-1].split('.')[-2]
            data = []
            for lbl in labels:
                if bbox_name in lbl:
                    number_cls = class_names.index(lbl.split('_')[-1].split('.')[-3])
                    with open(lbl,'r') as f:
                        while True:
                            line = f.readline()
                            if not line:
                                break
                            line = line.replace('0', str(number_cls), 1)
                            data.append(line)
            
            with open(os.path.join('./', split_type, bbox_name + '.txt'), 'w') as f:
                for d in data:
                    f.write(d)
        except IsADirectoryError:
            continue

In [35]:
seed = 50    
init_seeds(seed=seed)

In [36]:
pd.Series([file_name.split('_')[-1] for file_name in os.listdir(train_labels_path)]).value_counts()

head..txt        1741
human..txt       1740
car..txt         1647
face..txt         630
carplate..txt     246
dtype: int64

In [37]:
img_names = [os.path.join(train_image_path,img_name) for img_name in os.listdir(train_image_path)]
lbl_names = [os.path.join(train_labels_path,img_name) for img_name in os.listdir(train_labels_path)]

In [38]:
fold = 0
kf = KFold(n_splits = 5)
splits = list(kf.split(img_names))
train_idx = list(splits[fold][0])
val_idx = list(splits[fold][1])
len(train_idx), len(val_idx)

(1601, 401)

In [39]:
X_train_0 = list((itemgetter(*train_idx)(img_names)))
y_train_0 = list((itemgetter(*train_idx)(lbl_names)))
X_val_0 = list((itemgetter(*val_idx)(img_names)))
y_val_0 = list((itemgetter(*val_idx)(lbl_names)))
X_test_0 = 'None'

In [40]:
type_2_image = {'train': X_train_0,
                'val': X_val_0,
                'test': X_test_0}

image_2_type = {}
for key, values in type_2_image.items():
    for file_name in values:
        image_2_type[file_name] = key

In [41]:
aitrain_dataset = ["train: ./train" + "\n",
                 "val: ./val" + "\n",
                 "test: ./test" + "\n",
                 "nc: 5" + "\n\n",
                 "names: ['car', 'head', 'face', 'human', 'carplate']",]

aid_p = yolor_data_path / 'dataset.yaml'
with open(aid_p, 'w') as f:
    f.writelines(aitrain_dataset)

In [71]:
os.makedirs('./train', exist_ok=True)
os.makedirs('./test', exist_ok=True)
os.makedirs('./val',exist_ok=True)

In [72]:
copy_files(img_names, image_2_type, lbl_names)

100%|██████████████████████████████████████| 2002/2002 [00:04<00:00, 453.67it/s]


In [73]:
%cd /media/andrey/Новый том/depersonalization_object_detection/task/yolor/data

/media/andrey/Новый том/depersonalization_object_detection/task/yolor/data


In [74]:
@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [75]:
%%writetemplate coco.yaml

nc: 5
names: ['car', 'head', 'face', 'human', 'carplate',]

In [76]:
%%writetemplate coco.names

car 
head 
face 
human 
carplate

In [77]:
%%writetemplate hyp-yolor.yaml
  
lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.2  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 15.0  # image rotation (+/- deg)
translate: 0.5  # image translation (+/- fraction)
scale: 0.5  # image scale (+/- gain)
shear: 30.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.0  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 0.8  # image mosaic (probability)
mixup: 0.0  # image mixup (probability)

In [78]:
!ls

coco.names  dataset.yaml	    hyp.scratch.1280.yaml  hyp-yolor.yaml
coco.yaml   hyp.finetune.1280.yaml  hyp.scratch.s.yaml


In [79]:
%cd /media/andrey/Новый том/depersonalization_object_detection/task/yolor/

/media/andrey/Новый том/depersonalization_object_detection/task/yolor


In [80]:
!python train.py \
 --batch-size 4 \
 --img 1280 1280 \
 --data '{yolor_data_path}/dataset.yaml' \
 --cfg './models/yolor-d6.yaml' \
 --weights '{base_path}/task/yolor/yolor-d6.pt' \
 --device 0 \
 --name yolor_d6 \
 --hyp '{yolor_data_path}/hyp.finetune.1280.yaml' \
 --epochs 600

Using torch 1.7.1+cu110 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MB)

Namespace(weights='/media/andrey/Новый том/depersonalization_object_detection/task/yolor/yolor-d6.pt', cfg='./models/yolor-d6.yaml', data='/media/andrey/Новый том/depersonalization_object_detection/task/yolor/data/dataset.yaml', hyp='/media/andrey/Новый том/depersonalization_object_detection/task/yolor/data/hyp.finetune.1280.yaml', epochs=150, batch_size=4, img_size=[1280, 1280], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='0', multi_scale=False, single_cls=False, adam=False, sync_bn=False, local_rank=-1, log_imgs=16, workers=8, project='runs/train', name='yolor_d6', exist_ok=False, total_batch_size=4, world_size=1, global_rank=-1, save_dir='runs/train/yolor_d67')
Start Tensorboard with "tensorboard --logdir runs/train", view at http://localhost:6006/
Hyperparameters {'lr0': 0.01, 'lrf': 0.2, 'momentum': 0.937, 'weight


     Epoch   gpu_mem       box       obj       cls     total   targets  img_size
     5/149     20.7G   0.03269   0.02288  0.002139   0.05771        20      1280
               Class      Images     Targets           P           R      mAP@.5
                 all         401    3.66e+03       0.429       0.666       0.628       0.387

     Epoch   gpu_mem       box       obj       cls     total   targets  img_size
     6/149     20.7G    0.0313   0.02241   0.00202   0.05574        11      1280
               Class      Images     Targets           P           R      mAP@.5
                 all         401    3.66e+03       0.412       0.665       0.618       0.389

     Epoch   gpu_mem       box       obj       cls     total   targets  img_size
     7/149     20.7G    0.0299   0.02162  0.001788   0.05331        33      1280
               Class      Images     Targets           P           R      mAP@.5
                 all         401    3.66e+03       0.475       0.673       0.641  

               Class      Images     Targets           P           R      mAP@.5
                 all         401    3.66e+03       0.616       0.722       0.702       0.495

     Epoch   gpu_mem       box       obj       cls     total   targets  img_size
    54/149     20.7G   0.02053    0.0172 0.0009191   0.03865        43      1280
               Class      Images     Targets           P           R      mAP@.5
                 all         401    3.66e+03        0.61       0.707       0.693        0.49

     Epoch   gpu_mem       box       obj       cls     total   targets  img_size
    55/149     20.7G   0.02017   0.01697 0.0009376   0.03808         9      1280
               Class      Images     Targets           P           R      mAP@.5
                 all         401    3.66e+03         0.6       0.713       0.697       0.491

     Epoch   gpu_mem       box       obj       cls     total   targets  img_size
    56/149     20.7G   0.01934   0.01659 0.0008546   0.03678        17

In [10]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir '{base_path}/task/yolor/runs/train/yolor_d67'

In [11]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir '{base_path}/task/yolor/runs/train/yolor_d68'

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [12]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir '{base_path}/task/yolor/runs/train/yolor_d69'

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# Предсказания модели

In [3]:
%cd /media/andrey/Новый том/depersonalization_object_detection/task/yolor/

/media/andrey/Новый том/depersonalization_object_detection/task/yolor


In [4]:
base_path = Path('/media/andrey/Новый том/depersonalization_object_detection/')
test_image_path = base_path / 'test/'
model_path = base_path / 'task/yolor/runs/train/yolor_d69/weights/best_r.pt'
inference_path = base_path / 'inference/'

In [5]:
conf = 0.25
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda
NVIDIA GeForce RTX 3090
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [58]:
!python detect.py \
 --source '{test_image_path}' \
 --weights '{model_path}' \
 --project '{inference_path}' \
 --conf 0.25 \
 --iou 0.5 \
 --img-size 2560 \
 --device 0 \
 --save-txt \
 --save-conf

Namespace(weights=['/media/andrey/Новый том/depersonalization_object_detection/task/yolor/runs/train/yolor_d69/weights/best_r.pt'], source='/media/andrey/Новый том/depersonalization_object_detection/test', img_size=2560, conf_thres=0.25, iou_thres=0.5, device='0', view_img=False, save_txt=True, save_conf=True, classes=None, agnostic_nms=False, augment=False, update=False, project='/media/andrey/Новый том/depersonalization_object_detection/inference', name='exp', exist_ok=False)
Using torch 1.7.1+cu110 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MB)

Fusing layers... 
Model Summary: 751 layers, 150989120 parameters, 0 gradients
image 1/855 /media/andrey/Новый том/depersonalization_object_detection/test/0_23-11-2021_02-18-50_PM.jpg: 1728x2560 2 carplates, Done. (0.116s)
image 2/855 /media/andrey/Новый том/depersonalization_object_detection/test/0_23-11-2021_02-18-56_PM.jpg: 2176x2560 1 carplates, Done. (0.150s)
image 3/855 /media/andrey/Новый том/depersonalization_object_detection/test/0_23-11

image 54/855 /media/andrey/Новый том/depersonalization_object_detection/test/0_25-11-2021_08-34-03_AM.jpg: 1920x2560 1 carplates, Done. (0.131s)
image 55/855 /media/andrey/Новый том/depersonalization_object_detection/test/0_25-11-2021_08-50-08_AM.jpg: 1792x2560 1 carplates, Done. (0.126s)
image 56/855 /media/andrey/Новый том/depersonalization_object_detection/test/0_25-11-2021_09-27-14_AM.jpg: 1728x2560 3 carplates, Done. (0.117s)
image 57/855 /media/andrey/Новый том/depersonalization_object_detection/test/0_25-11-2021_09-46-26_AM.jpg: 1856x2560 1 carplates, Done. (0.122s)
image 58/855 /media/andrey/Новый том/depersonalization_object_detection/test/0_25-11-2021_10-03-07_AM.jpg: 1728x2560 1 carplates, Done. (0.115s)
image 59/855 /media/andrey/Новый том/depersonalization_object_detection/test/0_25-11-2021_10-03-50_AM.jpg: 1408x2560 1 carplates, Done. (0.098s)
image 60/855 /media/andrey/Новый том/depersonalization_object_detection/test/0_25-11-2021_10-55-30_AM.jpg: 2048x2560 1 carplates, 

image 111/855 /media/andrey/Новый том/depersonalization_object_detection/test/23-11-2021_02-36-28_PM.jpg: 1472x2560 13 cars, 2 heads, 2 humans, Done. (0.101s)
image 112/855 /media/andrey/Новый том/depersonalization_object_detection/test/23-11-2021_02-37-17_PM.jpg: 1472x2560 15 cars, 1 heads, 2 humans, Done. (0.100s)
image 113/855 /media/andrey/Новый том/depersonalization_object_detection/test/23-11-2021_02-37-22_PM.jpg: 1472x2560 13 cars, 1 heads, 1 faces, 1 humans, Done. (0.100s)
image 114/855 /media/andrey/Новый том/depersonalization_object_detection/test/23-11-2021_02-37-31_PM.jpg: 1472x2560 9 cars, 2 heads, 1 faces, 2 humans, Done. (0.102s)
image 115/855 /media/andrey/Новый том/depersonalization_object_detection/test/23-11-2021_02-37-35_PM.jpg: 1472x2560 8 cars, 2 heads, 1 humans, Done. (0.101s)
image 116/855 /media/andrey/Новый том/depersonalization_object_detection/test/23-11-2021_02-37-57_PM.jpg: 1472x2560 21 cars, 7 heads, 1 faces, 7 humans, Done. (0.101s)
image 117/855 /media/

image 162/855 /media/andrey/Новый том/depersonalization_object_detection/test/23-11-2021_05-01-36_PM.jpg: 1472x2560 2 cars, 1 heads, 2 humans, Done. (0.100s)
image 163/855 /media/andrey/Новый том/depersonalization_object_detection/test/23-11-2021_05-01-38_PM.jpg: 1472x2560 1 cars, 1 heads, 1 faces, 1 humans, Done. (0.104s)
image 164/855 /media/andrey/Новый том/depersonalization_object_detection/test/23-11-2021_05-02-13_PM.jpg: 1472x2560 6 cars, 2 heads, 2 humans, Done. (0.109s)
image 165/855 /media/andrey/Новый том/depersonalization_object_detection/test/23-11-2021_05-02-16_PM.jpg: 1472x2560 1 heads, 1 humans, Done. (0.104s)
image 166/855 /media/andrey/Новый том/depersonalization_object_detection/test/23-11-2021_05-18-06_PM.jpg: 1472x2560 15 cars, 1 humans, Done. (0.104s)
image 167/855 /media/andrey/Новый том/depersonalization_object_detection/test/23-11-2021_05-18-46_PM.jpg: 1472x2560 24 cars, 1 humans, Done. (0.103s)
image 168/855 /media/andrey/Новый том/depersonalization_object_dete

image 214/855 /media/andrey/Новый том/depersonalization_object_detection/test/24-11-2021_01-55-37_AM.jpg: 1472x2560 7 cars, 1 heads, 1 humans, Done. (0.104s)
image 215/855 /media/andrey/Новый том/depersonalization_object_detection/test/24-11-2021_02-06-04_PM.jpg: 1472x2560 2 cars, 2 heads, 2 humans, Done. (0.106s)
image 216/855 /media/andrey/Новый том/depersonalization_object_detection/test/24-11-2021_02-06-37_PM.jpg: 1472x2560 3 cars, 2 heads, 2 humans, Done. (0.106s)
image 217/855 /media/andrey/Новый том/depersonalization_object_detection/test/24-11-2021_02-06-55_PM.jpg: 1472x2560 10 cars, 3 heads, 3 humans, Done. (0.110s)
image 218/855 /media/andrey/Новый том/depersonalization_object_detection/test/24-11-2021_02-24-52_PM.jpg: 1472x2560 9 cars, 7 heads, 5 faces, 7 humans, Done. (0.106s)
image 219/855 /media/andrey/Новый том/depersonalization_object_detection/test/24-11-2021_02-25-19_PM.jpg: 1472x2560 18 cars, 7 heads, 7 humans, Done. (0.100s)
image 220/855 /media/andrey/Новый том/dep

...

image 625/855 /media/andrey/Новый том/depersonalization_object_detection/test/25-11-2021_08-16-40_AM.jpg: 1472x2560 13 cars, 6 heads, 3 faces, 5 humans, Done. (0.101s)
image 626/855 /media/andrey/Новый том/depersonalization_object_detection/test/25-11-2021_08-32-04_AM.jpg: 1472x2560 4 cars, Done. (0.101s)
image 627/855 /media/andrey/Новый том/depersonalization_object_detection/test/25-11-2021_08-32-41_AM.jpg: 1472x2560 15 cars, 3 heads, 2 faces, 2 humans, Done. (0.102s)
image 628/855 /media/andrey/Новый том/depersonalization_object_detection/test/25-11-2021_08-32-54_AM.jpg: 1472x2560 6 cars, 1 heads, 1 faces, 1 humans, Done. (0.101s)
image 629/855 /media/andrey/Новый том/depersonalization_object_detection/test/25-11-2021_08-33-28_AM.jpg: 1472x2560 12 cars, 3 heads, 3 humans, Done. (0.102s)
image 630/855 /media/andrey/Новый том/depersonalization_object_detection/test/25-11-2021_08-33-34_AM.jpg: 1472x2560 23 cars, 3 heads, 5 humans, Done. (0.101s)
image 631/855 /media/andrey/Новый том/de

image 676/855 /media/andrey/Новый том/depersonalization_object_detection/test/25-11-2021_10-38-15_AM.jpg: 1472x2560 20 cars, 4 heads, 1 faces, 4 humans, Done. (0.101s)
image 677/855 /media/andrey/Новый том/depersonalization_object_detection/test/25-11-2021_10-38-29_AM.jpg: 1472x2560 6 cars, 1 heads, 1 humans, Done. (0.101s)
image 678/855 /media/andrey/Новый том/depersonalization_object_detection/test/25-11-2021_10-38-33_AM.jpg: 1472x2560 5 cars, 3 heads, 1 faces, 2 humans, Done. (0.102s)
image 679/855 /media/andrey/Новый том/depersonalization_object_detection/test/25-11-2021_10-38-55_AM.jpg: 1472x2560 42 cars, 3 heads, 1 faces, 3 humans, Done. (0.101s)
image 680/855 /media/andrey/Новый том/depersonalization_object_detection/test/25-11-2021_10-39-17_AM.jpg: 1472x2560 13 cars, 1 heads, 1 humans, Done. (0.101s)
image 681/855 /media/andrey/Новый том/depersonalization_object_detection/test/25-11-2021_10-55-24_AM.jpg: 1472x2560 5 cars, 3 heads, 3 humans, Done. (0.101s)
image 682/855 /media/a

image 727/855 /media/andrey/Новый том/depersonalization_object_detection/test/26-11-2021_04-49-01_AM.jpg: 1472x2560 17 cars, 2 heads, 1 faces, 2 humans, 1 carplates, Done. (0.102s)
image 728/855 /media/andrey/Новый том/depersonalization_object_detection/test/26-11-2021_05-04-43_AM.jpg: 1472x2560 5 cars, 1 heads, 1 humans, Done. (0.102s)
image 729/855 /media/andrey/Новый том/depersonalization_object_detection/test/26-11-2021_05-04-56_AM.jpg: 1472x2560 15 cars, 2 heads, 2 humans, Done. (0.102s)
image 730/855 /media/andrey/Новый том/depersonalization_object_detection/test/26-11-2021_05-05-59_AM.jpg: 1472x2560 9 cars, 5 heads, 2 faces, 5 humans, Done. (0.101s)
image 731/855 /media/andrey/Новый том/depersonalization_object_detection/test/26-11-2021_05-23-05_AM.jpg: 1472x2560 8 cars, 2 heads, 1 faces, 2 humans, Done. (0.101s)
image 732/855 /media/andrey/Новый том/depersonalization_object_detection/test/26-11-2021_05-23-48_AM.jpg: 1472x2560 11 cars, 5 heads, 2 faces, 5 humans, Done. (0.101s)


image 778/855 /media/andrey/Новый том/depersonalization_object_detection/test/26-11-2021_08-04-15_AM.jpg: 1472x2560 9 cars, 3 heads, 2 faces, 3 humans, Done. (0.101s)
image 779/855 /media/andrey/Новый том/depersonalization_object_detection/test/26-11-2021_08-05-04_AM.jpg: 1472x2560 15 cars, 1 heads, 1 faces, 3 humans, Done. (0.102s)
image 780/855 /media/andrey/Новый том/depersonalization_object_detection/test/26-11-2021_08-05-07_AM.jpg: 1472x2560 35 cars, 2 heads, 3 humans, Done. (0.102s)
image 781/855 /media/andrey/Новый том/depersonalization_object_detection/test/26-11-2021_08-05-21_AM.jpg: 1472x2560 27 cars, 2 heads, 1 faces, 4 humans, Done. (0.102s)
image 782/855 /media/andrey/Новый том/depersonalization_object_detection/test/26-11-2021_08-06-03_AM.jpg: 1472x2560 1 cars, 5 heads, 3 humans, Done. (0.101s)
image 783/855 /media/andrey/Новый том/depersonalization_object_detection/test/26-11-2021_08-06-23_AM.jpg: 1472x2560 9 cars, Done. (0.101s)
image 784/855 /media/andrey/Новый том/dep

image 829/855 /media/andrey/Новый том/depersonalization_object_detection/test/26-11-2021_11-22-36_AM.jpg: 1472x2560 22 cars, 4 heads, 4 faces, 9 humans, Done. (0.102s)
image 830/855 /media/andrey/Новый том/depersonalization_object_detection/test/26-11-2021_11-25-09_AM.jpg: 1472x2560 15 cars, 2 heads, 2 faces, 2 humans, Done. (0.101s)
image 831/855 /media/andrey/Новый том/depersonalization_object_detection/test/26-11-2021_11-42-01_AM.jpg: 1472x2560 9 cars, 5 heads, 5 faces, 5 humans, Done. (0.102s)
image 832/855 /media/andrey/Новый том/depersonalization_object_detection/test/26-11-2021_11-42-06_AM.jpg: 1472x2560 4 cars, 2 heads, 1 faces, 2 humans, Done. (0.102s)
image 833/855 /media/andrey/Новый том/depersonalization_object_detection/test/26-11-2021_11-44-10_AM.jpg: 1472x2560 1 cars, 1 heads, 2 humans, Done. (0.101s)
image 834/855 /media/andrey/Новый том/depersonalization_object_detection/test/26-11-2021_11-44-35_AM.jpg: 1472x2560 19 cars, 5 heads, 3 faces, 6 humans, Done. (0.101s)
imag

In [6]:
labels_path = inference_path / 'exp17/labels'

In [7]:
def get_soliton(path_to_txt_folder):
    simple_solution = []
    for detection_file in os.listdir(path_to_txt_folder):
        img_name = detection_file.split('.')[0] + '.jpg'
        with open(os.path.join(path_to_txt_folder, detection_file), 'r') as f:
            data = f.read()
            data = [i for i in data.split('\n') if i != '']
        for line in data:
            val = [float(i) for i in line.split()]
            cls, xywh, conf = int(val[0]), val[1:5], val[5]
            
            center_x, center_y, width, height = xywh
            xmin = center_x - (width / 2)
            xmax = center_x + (width / 2)
            ymin = center_y - (height / 2)
            ymax = center_y + (height / 2)
            simple_solution.append([img_name, cls, conf, xmin, xmax, ymin, ymax])
    return simple_solution

In [8]:
%cd /media/andrey/Новый том/depersonalization_object_detection/task/

/media/andrey/Новый том/depersonalization_object_detection/task


In [9]:
solution = get_soliton(labels_path)
solution = pd.DataFrame(solution, columns=['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])
solution.to_csv('sample_solution.csv', sep=';', index=False)